<a href="https://colab.research.google.com/github/Mostafa-Hamidifard/cvx-optimization/blob/main/cvx_first_project_extracting_neural_signals_from_an_ephys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>